<a href="https://colab.research.google.com/github/Tifou-Kerdjou/deep-learning/blob/main/KNN_Naive_Bayes_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , confusion_matrix , f1_score, recall_score
from scipy.spatial import distance
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import mode
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC as svm

# KNN

In [ ]:
X, Y = load_iris(return_X_y=True)

x_train,x_test, y_train, y_test = train_test_split(X, Y, test_size = .3, shuffle=True)


def eucledian(p1,p2):
    dist = np.sqrt(np.sum((p1-p2)**2))
    return dist
class KNN():
    def fit( X_train, Y_train):
        X_train = X_train
        Y_train = Y_train
    #Euclidean Distance

    
    #Function to calculate KNN
    def predict(x_input,y , k):
        op_labels = []
        
        #Loop through the Datapoints to be classified
        for item in x_input: 
            
            #Array to store distances
            point_dist = []
            
            #Loop through each training Data
            for j in range(len(x_train)): 
                distances = eucledian(np.array(x_train[j,:]) , item) 
                #Calculating the distance
                point_dist.append(distances) 
            point_dist = np.array(point_dist) 
            
            #Sorting the array while preserving the index
            #Keeping the first K datapoints
            dist = np.argsort(point_dist)[:k]
            
            #Labels of the K datapoints from above
            labels = y[dist]
            
            #Majority voting
            lab = mode(labels) 
            lab = lab.mode[0]
            op_labels.append(lab)
    
        return op_labels

In [ ]:
k = int(input("Enter k: "))
KNN.fit(x_train,y_train)
predictions= KNN.predict(x_test,y_train,k)


print("Fitting classifier...")
#classifier.fit(x_train, y_train)
print("Successfully fitted classifier")
print("Making predictions")
#predictions = classifier.predict(x_test)
print("Completed making predictions")
from sklearn.metrics import accuracy_score
print ("Accuracy:", accuracy_score(y_test, predictions)*100, "%")

Enter k: 3
Fitting classifier...
Successfully fitted classifier
Making predictions
Completed making predictions
Accuracy: 97.77777777777777 %


prédéfini

In [ ]:
k = int(input("Enter k: "))
KNNp=KNeighborsClassifier(k)
KNNp.fit(x_train,y_train)
predictions= KNNp.predict(x_test)


print("Fitting classifier...")
#classifier.fit(x_train, y_train)
print("Successfully fitted classifier")
print("Making predictions")
#predictions = classifier.predict(x_test)
print("Completed making predictions")
from sklearn.metrics import accuracy_score
print ("Accuracy:", accuracy_score(y_test, predictions)*100, "%")

Enter k: 3
Fitting classifier...
Successfully fitted classifier
Making predictions
Completed making predictions
Accuracy: 97.77777777777777 %


### 10 cross validation

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
cv = KFold(n_splits=10, random_state=1, shuffle=True)
model = KNeighborsClassifier(3)
scores = cross_val_score(model, X, Y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (np.mean(scores)*100, np.std(scores)))

Accuracy: 96.667 (0.045)


In [ ]:
confusion_matrix(y_test,predictions)



array([[11,  0,  0],
       [ 0, 18,  0],
       [ 0,  1, 15]])

In [ ]:
f1_score(y_test,predictions,average='weighted')

0.97771965513901

In [ ]:
recall_score(y_test,predictions,average='weighted')

0.9777777777777777

# Naive Bayes

In [ ]:
class NaiveBayesClassifier():
    '''
    Bayes Theorem form
    P(y|X) = P(X|y) * P(y) / P(X)
    '''
    def calc_prior(self, features, target):
        '''
        prior probability P(y)
        calculate prior probabilities
        '''
        self.prior = (features.groupby(target).apply(lambda x: len(x)) / self.rows).to_numpy()

        return self.prior
    
    def calc_statistics(self, features, target):
        '''
        calculate mean, variance for each column and convert to numpy array
        ''' 
        self.mean = features.groupby(target).apply(np.mean).to_numpy()
        self.var = features.groupby(target).apply(np.var).to_numpy()
              
        return self.mean, self.var
    
    def gaussian_density(self, class_idx, x):     
        '''
        calculate probability from gaussian density function (normally distributed)
        we will assume that probability of specific target value given specific class is normally distributed 
        
        probability density function derived from wikipedia:
        (1/√2pi*σ) * exp((-1/2)*((x-μ)^2)/(2*σ²)), where μ is mean, σ² is variance, σ is quare root of variance (standard deviation)
        '''
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp((-1/2)*((x-mean)**2) / (2 * var))
#         numerator = np.exp(-((x-mean)**2 / (2 * var)))
        denominator = np.sqrt(2 * np.pi * var)
        prob = numerator / denominator
        return prob
    def calc_posterior(self, x):
        posteriors = []

        # calculate posterior probability for each class
        for i in range(self.count):
            prior = np.log(self.prior[i]) ## use the log to make it more numerically stable
            conditional = np.sum(np.log(self.gaussian_density(i, x))) # use the log to make it more numerically stable
            posterior = prior + conditional
            posteriors.append(posterior)
        # return class with highest posterior probability
        return self.classes[np.argmax(posteriors)]
     

    def fit(self, features, target):
        self.classes = np.unique(target)
        self.count = len(self.classes)
        self.feature_nums = features.shape[1]
        self.rows = features.shape[0]
        
        self.calc_statistics(features, target)
        self.calc_prior(features, target)
        
    def predict(self, features):
        preds = [self.calc_posterior(f) for f in features.to_numpy()]
        return preds

    def accuracy(self, y_test, y_pred):
        accuracy = np.sum(y_test == y_pred) / len(y_test)
        return accuracy

In [ ]:
# upload Iris dataset -  shape is (150, 5)
df = pd.read_csv("iris.data" ,  sep=",")
# shuffle dataset with sample
df = df.sample(frac=1, random_state=1).reset_index(drop=True)
# df shape
print(df.shape)
# set features and target
X, y = df.iloc[:, :-1], df.iloc[:, -1]


# # split on train and test 0.7/0.3
X_train, X_test, y_train, y_test = X[:100], X[100:], y[:100], y[100:]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(149, 5)
(100, 4) (100,)
(49, 4) (49,)


In [ ]:
x = NaiveBayesClassifier()

x.fit(X_train, y_train)
predictions = x.predict(X_test)
x.accuracy(y_test, predictions)
confusion_matrix(y_test, predictions)



array([[17,  0,  0],
       [ 0, 18,  1],
       [ 0,  1, 12]])

In [ ]:
f1_score(y_test,predictions,average='weighted')

0.9591836734693877

In [ ]:
recall_score(y_test,predictions,average='weighted')

0.9591836734693877

## prédéfini

In [ ]:
clf = GaussianNB()
clf.fit(X_train, y_train)
predictionsPré = clf.predict(X_test)
confusion_matrix(y_test, predictionsPré)



array([[17,  0,  0],
       [ 0, 18,  1],
       [ 0,  1, 12]])

In [ ]:
f1_score(y_test,predictionsPré,average='weighted')

0.9591836734693877

In [ ]:
recall_score(y_test,predictionsPré,average='weighted')

0.9591836734693877

### 10 cross validation

In [ ]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)
model = GaussianNB()
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (np.mean(scores)*100, np.std(scores)))

Accuracy: 94.667 (0.058)


# SVM

In [ ]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)
model=svm(gamma='auto')
X, Y = load_iris(return_X_y=True)
x_train,x_test, y_train, y_test = train_test_split(X, Y, test_size = .3, shuffle=True)
scores = cross_val_score(model, X, Y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (np.mean(scores)*100, np.std(scores)))



model.fit(x_train,y_train,None)
y_pre=model.predict(x_test)
confusion_matrix(y_test,y_pre)


Accuracy: 97.333 (0.033)


array([[14,  0,  0],
       [ 0, 16,  0],
       [ 0,  0, 15]])

In [ ]:
f1_score(y_test,y_pre,average='weighted')

1.0

In [ ]:

recall_score(y_test,y_pre,average='weighted')

1.0